# Capturando datos de la web de la AEMET
Como complemento al termostado que estoy fabricando sobre la Raspberry me encuentro en la necesidad de captar la temperatura exterior. Una forma rápida y sencilla esdesde la web de la [AEMET] ya que es posible descargar un archivo csv y de ello nos vamos a aprovechar...

En principio, todo lo que necesito es la temperatura exterior actual, la hora a la que ha sido tomada y la media diaria de temperatura. He dado con un par de métodos para obtener estas mediciones, uno trabajando los datos en bruto y otro usando la librería de análisis de datos [Pandas].

## Método 1 (Modo Manual)

[AEMET]: http://www.aemet.es/es/eltiempo/observacion/ultimosdatos_9434P_datos-horarios.csv?k=arn&l=9434P&datos=det&w=0&f=temperatura&x=h24
[Pandas]: https://pandas.pydata.org

In [5]:
import requests
from datetime import datetime

url = "http://www.aemet.es/es/eltiempo/observacion/ultimosdatos_9434P_datos-horarios.csv?k=arn&l=9434P&datos=det&w=0&f=temperatura&x=h24"
datos = requests.get(url, allow_redirects=True).text.replace('"', "").splitlines()

aemet_hora = datetime.now().strftime('%Y/%m/%d %H:%M')
aemet_temp = float(datos[4].split(",")[1])

temp = 0.0

for i in range(4,len(datos)):
    temp += float(datos[i].split(",")[1])
    
aemet_media = round(temp / (i-3),1)

print(f"{aemet_hora} - Se ha tomado una Tª de {aemet_temp}ºC, la media diaria es de {aemet_media}ºC")

2021/01/29 19:17 - Se ha tomado una Tª de 17.7ºC, la media diaria es de 15.7ºC


El script sigue los siguientes pasos:
- Petición al servidor
- Eliminación de las dobles comillas
- Conversión en una lista por líneas
- Generación de la hora actual mediante la clase "Datetime"
- Creación de una lista a partir de la 4ª fila donde está el último dato y quedarnos con el segundo valor (Tª)
- Cálculo de la media de Tª mediante un bucle for que recorre los registros (Desde la 4ª fila hasta el final)

Bastante sencillo y rápido

## Método 2 (Modo Pandas)

In [32]:
import pandas as pd

url = "http://www.aemet.es/es/eltiempo/observacion/ultimosdatos_9434P_datos-horarios.csv?k=arn&l=9434P&datos=det&w=0&f=temperatura&x=h24"
df = pd.read_csv(url,encoding = "ISO-8859-1",skiprows=3)

campos = df.columns.values

hora = campos[0]
temp = campos[1]

aemet_hora = datetime.strptime(df.iloc[0][hora], '%d/%m/%Y %H:%M')
aemet_media = round(df[temp].mean(),1)
aemet_temp = df.iloc[0][campos[1]]

print(f"{aemet_hora} - Se ha tomado una Tª de {aemet_temp}ºC, la media diaria es de {aemet_media}ºC")

2021-01-29 18:00:00 - Se ha tomado una Tª de 17.7ºC, la media diaria es de 15.7ºC


El script sigue los siguientes pasos:
- Creación de un "dataframe" a partir de una url cuyo contenido está codificado en "ISO-8859-1" y del que despreciamos las 3 primeras filas
- Extracción de los encabezados
- Asignación de una variable a cada encabezado
- Extracción de la hora mediante "iloc" y cambio de formato mediante datetime
- Cálculo de la media de la columna de temperaturas
- Extracción de la ultima temperatura mediante "iloc"